In [ ]:
# import
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# IMPORTS

# preprocessing
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore') 
from scipy import stats
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler


# models
import xgboost as xgb
import lightgbm as lgb

# sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score

In [ ]:
train = pd.read_csv('/content/gdrive/MyDrive/vehicle/train.csv')
test = pd.read_csv('/content/gdrive/MyDrive/vehicle/test.csv')
train.head()

,UNIQUEID,DISBURSED_AMOUNT,ASSET_COST,LTV,BRANCH_ID,SUPPLIER_ID,MANUFACTURER_ID,CURRENT_PINCODE_ID,DATE_OF_BIRTH,EMPLOYMENT_TYPE,DISBURSAL_DATE,STATE_ID,EMPLOYEE_CODE_ID,MOBILENO_AVL_FLAG,AADHAR_FLAG,PAN_FLAG,VOTERID_FLAG,DRIVING_FLAG,PASSPORT_FLAG,PERFORM_CNS_SCORE,PERFORM_CNS_SCORE_DESCRIPTION,PRI_NO_OF_ACCTS,PRI_ACTIVE_ACCTS,PRI_OVERDUE_ACCTS,PRI_CURRENT_BALANCE,PRI_SANCTIONED_AMOUNT,PRI_DISBURSED_AMOUNT,SEC_NO_OF_ACCTS,SEC_ACTIVE_ACCTS,SEC_OVERDUE_ACCTS,SEC_CURRENT_BALANCE,SEC_SANCTIONED_AMOUNT,SEC_DISBURSED_AMOUNT,PRIMARY_INSTAL_AMT,SEC_INSTAL_AMT,NEW_ACCTS_IN_LAST_SIX_MONTHS,DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS,AVERAGE_ACCT_AGE,CREDIT_HISTORY_LENGTH,NO_OF_INQUIRIES,LOAN_DEFAULT
0,420825,50578,58400,89.55,67,22807,45,1441,01-01-1984,Salaried,03-08-2018,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
1,537409,47145,65550,73.23,67,22807,45,1502,31-07-1985,Self employed,26-09-2018,6,1998,1,1,0,0,0,0,598,I-Medium Risk,1,1,1,27600,50200,50200,0,0,0,0,0,0,1991,0,0,1,1yrs 11mon,1yrs 11mon,0,1
2,417566,53278,61360,89.63,67,22807,45,1497,24-08-1985,Self employed,01-08-2018,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
3,624493,57513,66113,88.48,67,22807,45,1501,30-12-1993,Self employed,26-10-2018,6,1998,1,1,0,0,0,0,305,L-Very High Risk,3,0,0,0,0,0,0,0,0,0,0,0,31,0,0,0,0yrs 8mon,1yrs 3mon,1,1
4,539055,52378,60300,88.39,67,22807,45,1495,09-12-1977,Self employed,26-09-2018,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,1,1


# PREPROCESSING

Missing values Interpolation

In [ ]:
train.isna().sum().unique()

In [ ]:
train.EMPLOYMENT_TYPE = train.EMPLOYMENT_TYPE.fillna("Missing")
test.EMPLOYMENT_TYPE = test.EMPLOYMENT_TYPE.fillna("Missing")
train_null_unique = train.EMPLOYMENT_TYPE.unique()
test_null_unique = test.EMPLOYMENT_TYPE.unique()

Transformation of some columns

In [ ]:
# Transofmation of AVERAGE_ACCT_AGE & CREDIT_HISTORY_LENGTH on train and test datasets:

train[['AVERAGE_ACCT_Yr','AVERAGE_ACCT_Month']] = train['AVERAGE_ACCT_AGE'].str.split("yrs",expand=True)
train[['AVERAGE_ACCT_Month','AVERAGE_ACCT_Month1']] = train['AVERAGE_ACCT_Month'].str.split("mon",expand=True)
train["AVERAGE_ACCT_AGE"] = train["AVERAGE_ACCT_Yr"].astype(str).astype(int)+((train["AVERAGE_ACCT_Month"].astype(str).astype(int))/12)
train= train.drop(columns=["AVERAGE_ACCT_Yr","AVERAGE_ACCT_Month",'AVERAGE_ACCT_Month1'])

test[['AVERAGE_ACCT_Yr','AVERAGE_ACCT_Month']] = test['AVERAGE_ACCT_AGE'].str.split("yrs",expand=True)
test[['AVERAGE_ACCT_Month','AVERAGE_ACCT_Month1']] = test['AVERAGE_ACCT_Month'].str.split("mon",expand=True)
test["AVERAGE_ACCT_AGE"]= test["AVERAGE_ACCT_Yr"].astype(str).astype(int)+((test["AVERAGE_ACCT_Month"].astype(str).astype(int))/12)
test= test.drop(columns= ["AVERAGE_ACCT_Yr","AVERAGE_ACCT_Month",'AVERAGE_ACCT_Month1'])

train[['CREDIT_HISTORY_LENGTH_Yr','CREDIT_HISTORY_LENGTH_Month']] = train['CREDIT_HISTORY_LENGTH'].str.split("yrs",expand=True)
train[['CREDIT_HISTORY_LENGTH_Month','CREDIT_HISTORY_LENGTH_Month1']] = train['CREDIT_HISTORY_LENGTH_Month'].str.split("mon",expand=True)
train["CREDIT_HISTORY_LENGTH"]= train["CREDIT_HISTORY_LENGTH_Yr"].astype(str).astype(int)+((train["CREDIT_HISTORY_LENGTH_Month"].astype(str).astype(int))/12)
train= train.drop(columns= ["CREDIT_HISTORY_LENGTH_Yr","CREDIT_HISTORY_LENGTH_Month",'CREDIT_HISTORY_LENGTH_Month1'])

test[['CREDIT_HISTORY_LENGTH_Yr','CREDIT_HISTORY_LENGTH_Month']] = test['CREDIT_HISTORY_LENGTH'].str.split("yrs",expand=True)
test[['CREDIT_HISTORY_LENGTH_Month','CREDIT_HISTORY_LENGTH_Month1']] = test['CREDIT_HISTORY_LENGTH_Month'].str.split("mon",expand=True)
test["CREDIT_HISTORY_LENGTH"]= test["CREDIT_HISTORY_LENGTH_Yr"].astype(str).astype(int)+((test["CREDIT_HISTORY_LENGTH_Month"].astype(str).astype(int))/12)
test= test.drop(columns= ["CREDIT_HISTORY_LENGTH_Yr","CREDIT_HISTORY_LENGTH_Month",'CREDIT_HISTORY_LENGTH_Month1'])

train['DATE_OF_BIRTH'] =  pd.to_datetime(train['DATE_OF_BIRTH'], format='%d-%m-%Y')
test['DATE_OF_BIRTH'] =  pd.to_datetime(test['DATE_OF_BIRTH'], format='%d-%m-%Y')
train['DISBURSAL_DATE'] =  pd.to_datetime(train['DISBURSAL_DATE'], format='%d-%m-%Y')
test['DISBURSAL_DATE'] =  pd.to_datetime(test['DISBURSAL_DATE'], format='%d-%m-%Y')

# Outlier Detection

In [ ]:
# number of observations in column
obs = len(train.DISBURSED_AMOUNT)

# calculate summary statistics
data_mean, data_std = train.DISBURSED_AMOUNT.mean(), train.DISBURSED_AMOUNT.std()
print(data_mean, data_std)

# z-score
cut_off = data_std * 3
lower, upper = data_mean - cut_off, data_mean + cut_off
outliers = [x for x in train.DISBURSED_AMOUNT if x < lower or x > upper]

print('outliers based on z-score', len(outliers))

No. of observations in column:  233154
Statistics: Mean=54356.994, Std dev=12971.286
Identified outliers: 3076


Label Encoding based on quartiles

In [ ]:
labels = ['Low', 'Medium', 'High', 'Extreme']
train['DISBURSED_AMOUNT_bins'] = pd.qcut(train['DISBURSED_AMOUNT'],
                              q=[0, .25, .5, .75, 1],
                              labels=labels)

Medium     58676
Low        58537
Extreme    58207
High       57734
Name: DISBURSED_AMOUNT_bins, dtype: int64

In [ ]:
bin_labels = ['Low', 'Medium', 'High', 'Extreme']
train['ASSET_COST_bins'] = pd.qcut(train['ASSET_COST'],
                              q=[0, .25, .5, .75, 1],
                              labels=bin_labels)

In [ ]:
bin_labels = ['Low', 'Medium', 'High', 'Extreme']
train['LTV_bins'] = pd.qcut(train['LTV'],
                              q=[0, .25, .5, .75, 1],
                              labels=bin_labels)

In [ ]:
bin_labels = ["No History",'Very Low', "Low" ,'Medium', 'High']
cut_bins = [-1,10,150, 350, 650, 1000]

train['PERFORM_CNS_SCORE_bins'] = pd.cut(train['PERFORM_CNS_SCORE'],
                              bins=cut_bins,
                              labels=bin_labels)

In [ ]:
bin_labels = ["One",'More than One']
cut_bins = [-1,1, 1000]

train['PRI_NO_OF_ACCTS_bins'] = pd.cut(train['PRI_NO_OF_ACCTS'],
                              bins=cut_bins,
                              labels=bin_labels)

In [ ]:
bin_labels = ["None",'One (or more)']
cut_bins = [-1,0, 1000]

train['PRI_OVERDUE_ACCTS_bins'] = pd.cut(train['PRI_OVERDUE_ACCTS'],
                              bins=cut_bins,
                              labels=bin_labels)

# Outlier Transformation

In [ ]:
def outlier_trans(x:float):
    if x <= lower:
        return(data_mean)
    elif x>= (upper):
        return(data_mean)
    else:
        return(x)
train["DISBURSED_AMOUNT_new"] = train["DISBURSED_AMOUNT"].apply(outlier_trans)

In [ ]:
train["ASSET_COST_new"]= train["ASSET_COST"].apply(outlier_trans)
train["LTV_new"]= train["LTV"].apply(outlier_trans)
train["PRI_NO_OF_ACCTS_new"]= train["PRI_NO_OF_ACCTS"].apply(outlier_trans)
train["PRI_OVERDUE_ACCTS_new"]= train["PRI_OVERDUE_ACCTS"].apply(outlier_trans)

Add age as a single feature

In [ ]:
now = pd.Timestamp('now')
train['age'] = (now - train['DATE_OF_BIRTH'])  

train['age']= train['age'].astype(str)
train[['age','age_waste']] = train['age'].str.split("days",expand=True)
train['age']= train['age'].astype(str).astype(int)
train= train.drop(columns= ['age_waste'])

0    14270
1    13693
2    13669
3    10619
4    16484
Name: age, dtype: int64


Add time of disbursal as a single feature

In [ ]:
train['disbursal_time'] = (now - train['DISBURSAL_DATE'])  

train['disbursal_time']= train['disbursal_time'].astype(str)
train[['disbursal_time','disbursal_time_waste']] = train['disbursal_time'].str.split("days",expand=True)
train['disbursal_time']= train['disbursal_time'].astype(str).astype(int)
train= train.drop(columns= ['disbursal_time_waste'])

0    1637
1    1583
2    1639
3    1553
4    1583
Name: disbursal_time, dtype: int64


# Feature Selection

Drop usless features (id's)

In [ ]:
train = train.drop([ 'DATE_OF_BIRTH' , 'STATE_ID', 'EMPLOYEE_CODE_ID', 'SUPPLIER_ID', 'MANUFACTURER_ID', 'CURRENT_PINCODE_ID','BRANCH_ID'],axis=1)

split on continuous and categorical variables.

In [ ]:
train_con = train[['EMPLOYMENT_TYPE', 'MOBILENO_AVL_FLAG', 'AADHAR_FLAG', 'PAN_FLAG', 'VOTERID_FLAG',
                   'DRIVING_FLAG', 'PASSPORT_FLAG', 'PERFORM_CNS_SCORE', 'PERFORM_CNS_SCORE_DESCRIPTION', 
                   'PRI_ACTIVE_ACCTS', 'PRI_CURRENT_BALANCE', 'PRI_DISBURSED_AMOUNT', 'SEC_NO_OF_ACCTS', 
                   'SEC_OVERDUE_ACCTS', 'SEC_CURRENT_BALANCE', 'PRIMARY_INSTAL_AMT', 'SEC_INSTAL_AMT', 
                   'DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS', 'CREDIT_HISTORY_LENGTH', 'NO_OF_INQUIRIES',
                   'LOAN_DEFAULT', 'DISBURSED_AMOUNT_new', 'ASSET_COST_new', 
                   'LTV_new', 'age', 'disbursal_time']]

In [ ]:
train_cat = train[['UNIQUEID', 'EMPLOYMENT_TYPE', 'MOBILENO_AVL_FLAG', 'AADHAR_FLAG', 'PAN_FLAG', 
                    'VOTERID_FLAG', 'DRIVING_FLAG', 'PASSPORT_FLAG', 'PERFORM_CNS_SCORE', 
                    'PERFORM_CNS_SCORE_DESCRIPTION', 'PRI_ACTIVE_ACCTS', 'PRI_CURRENT_BALANCE',
                    'PRI_DISBURSED_AMOUNT', 'SEC_NO_OF_ACCTS', 'SEC_OVERDUE_ACCTS', 'SEC_CURRENT_BALANCE',
                    'PRIMARY_INSTAL_AMT', 'SEC_INSTAL_AMT', 'DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS', 
                    'CREDIT_HISTORY_LENGTH', 'NO_OF_INQUIRIES', 'LOAN_DEFAULT',
                    'DISBURSED_AMOUNT_bins', 'ASSET_COST_bins', 'LTV_bins',
                    'PERFORM_CNS_SCORE_bins', 'PRI_NO_OF_ACCTS_bins', 'PRI_OVERDUE_ACCTS_bins', 'age', 'disbursal_time']]

data scaling

In [ ]:
scaler = StandardScaler() 

def scaling(df:pd.DataFrame, cols_to_scale:pd.Series):

    for col in cols_to_scale:

        df[col] = pd.DataFrame(scaler.fit_transform(pd.DataFrame(train_con[col])),columns=[col])

    return df

In [ ]:
scaled_df = scaleColumns(train_con,['PERFORM_CNS_SCORE','PRI_ACTIVE_ACCTS', 'PRI_CURRENT_BALANCE', 
                                 'PRI_DISBURSED_AMOUNT', 'SEC_NO_OF_ACCTS', 'SEC_OVERDUE_ACCTS', 
                                 'SEC_CURRENT_BALANCE', 'PRIMARY_INSTAL_AMT', 'SEC_INSTAL_AMT',
                                 'DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS', 'CREDIT_HISTORY_LENGTH', 
                                 'NO_OF_INQUIRIES', 'DISBURSED_AMOUNT_new',
                                 'ASSET_COST_new', 'LTV_new', 'age', 'disbursal_time'])

scaled_df.head()

,EMPLOYMENT_TYPE,MOBILENO_AVL_FLAG,AADHAR_FLAG,PAN_FLAG,VOTERID_FLAG,DRIVING_FLAG,PASSPORT_FLAG,PERFORM_CNS_SCORE,PERFORM_CNS_SCORE_DESCRIPTION,PRI_ACTIVE_ACCTS,PRI_CURRENT_BALANCE,PRI_DISBURSED_AMOUNT,SEC_NO_OF_ACCTS,SEC_OVERDUE_ACCTS,SEC_CURRENT_BALANCE,PRIMARY_INSTAL_AMT,SEC_INSTAL_AMT,DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS,CREDIT_HISTORY_LENGTH,NO_OF_INQUIRIES,LOAN_DEFAULT,DISBURSED_AMOUNT_new,ASSET_COST_new,LTV_new,age,disbursal_time
0,Salaried,1,1,0,0,0,0,-0.855453,No Bureau History Available,-0.535617,-0.176064,-0.091711,-0.094259,-0.065216,-0.031884,-0.086581,-0.020784,-0.253566,-0.568640,-0.292450,0,-0.282591,-1.002057,-3.637979e-11,0.026250,1.880044
1,Self employed,1,1,0,0,0,0,0.911822,I-Medium Risk,-0.020549,-0.146773,-0.070599,-0.094259,-0.065216,-0.031884,-0.073427,-0.020784,2.347632,0.236085,-0.292450,1,-0.589728,-0.257144,-3.637979e-11,-0.134554,-0.094516
2,Self employed,1,1,0,0,0,0,-0.855453,No Bureau History Available,-0.535617,-0.176064,-0.091711,-0.094259,-0.065216,-0.031884,-0.086581,-0.020784,-0.253566,-0.568640,-0.292450,0,-0.041033,-0.693673,-3.637979e-11,-0.141243,1.953176
3,Self employed,1,1,0,0,0,0,0.045917,L-Very High Risk,-0.535617,-0.176064,-0.091711,-0.094259,-0.065216,-0.031884,-0.086376,-0.020784,-0.253566,-0.043819,1.122986,1,0.337856,-0.198489,-3.637979e-11,-0.991244,-1.191493
4,Self employed,1,1,0,0,0,0,-0.855453,No Bureau History Available,-0.535617,-0.176064,-0.091711,-0.094259,-0.065216,-0.031884,-0.086581,-0.020784,-0.253566,-0.568640,1.122986,1,-0.121552,-0.804108,-3.637979e-11,0.643267,-0.094516


get dummies - transform dummies

In [ ]:
train_dummy = pd.get_dummies(scaled_df, prefix_sep='_', drop_first=True)
train_dummy.head()

y = train_dummy[['LOAN_DEFAULT']]
X = train_dummy.loc[:, train_dummy.columns != 'LOAN_DEFAULT']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
k_fold = KFold(n_splits=10, shuffle=True, random_state=0)

# MODELS

Random Forests

In [ ]:
# fitting
rf = RandomForestClassifier(n_estimators=10).fit(X_train, y_train)

# predicting
rf_pred = rf.predict(X_test)
best_acc_rf = (cross_val_score(rf, X_train, y_train, cv=k_fold, n_jobs=1, scoring = 'accuracy').mean())
print("Balanced Accuracy Score ",balanced_accuracy_score(y_test, rf_pred))
print("Best Accuracy Score ",best_acc_rf))

[[35090  1415]
 [ 9439   687]]
77.0


XGBoost

In [ ]:
# fitting
xgb_cls = xgb.XGBClassifier(n_estimators=10).fit(X_train, y_train)

# predicting
xgb_pred = xgb_cls.predict(X_test)
best_acc_xgb = (cross_val_score(xgb_cls, X_train, y_train, cv=k_fold, n_jobs=1, scoring = 'accuracy').mean())
print("Balanced Accuracy Score ",balanced_accuracy_score(y_test, xgb_pred))
print("Best Accuracy Score ",best_acc_xgb))

LightGBM

In [ ]:
# fitting
lgb_cls = lgb.LGBMClassifier(n_estimators=10).fit(X_train, y_train)

# predicting
lgb_pred = lgb_cls.predict(X_test)
best_acc_lgb = (cross_val_score(lgb_cls, X_train, y_train, cv=k_fold, n_jobs=1, scoring = 'accuracy').mean())
print("Balanced Accuracy Score ",balanced_accuracy_score(y_test, lgb_pred))
print("Best Accuracy Score ",best_acc_lgb))

# Save best model to a pickle format

In [ ]:
import pickle
pickle.dump(rfc, open('manou_model_2.pkl', 'wb'))